<a href="https://colab.research.google.com/github/CalinRusu95/parltrack-data-fetcher/blob/main/fetch_parltrack_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install requests zstandard

In [ ]:
import requests
import zstandard as zstd
import json
import os
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
# Constants
CREDENTIALS_PATH = "/tmp/credentials.json"
LOG_FILE_PATH = f"/tmp/fetch_and_upload_log_{datetime.now().strftime('%Y%m%d')}.txt"
DRIVE_FOLDER_ID = "16QmR40u_BF8K_DCSttSEOxYidzsuEkTU"  # Replace with your Google Drive folder ID
PARLTRACK_DUMPS_URL = "https://parltrack.eu/dumps/"

In [ ]:
# Write credentials from the secret
def write_credentials(secret):
    with open(CREDENTIALS_PATH, "w") as f:
        f.write(secret)

# Authenticate with Google Drive API
def authenticate_google_drive():
    credentials = Credentials.from_service_account_file(
        CREDENTIALS_PATH,
        scopes=["https://www.googleapis.com/auth/drive"]
    )
    service = build("drive", "v3", credentials=credentials)
    return service

# Upload file to Google Drive
def find_file_by_name(service, file_name, folder_id):
    """
    Find a file in Google Drive by its name within a specific folder.
    """
    query = f"'{folder_id}' in parents and name='{file_name}' and trashed=false"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    return files[0] if files else None

def upload_to_drive(service, file_path, file_name):
    """
    Uploads a file to Google Drive. Updates the file if it already exists.
    """
    try:
        # Check if the file already exists
        existing_file = find_file_by_name(service, file_name, DRIVE_FOLDER_ID)
        if existing_file:
            print(f"File '{file_name}' already exists. Updating the existing file (ID: {existing_file['id']}).")
            media = MediaFileUpload(file_path, resumable=True)
            updated_file = service.files().update(
                fileId=existing_file['id'], media_body=media
            ).execute()
            print(f"Updated file ID: {updated_file.get('id')}")
        else:
            # Create a new file
            print(f"File '{file_name}' does not exist. Creating a new file.")
            file_metadata = {
                "name": file_name,
                "parents": [DRIVE_FOLDER_ID]
            }
            media = MediaFileUpload(file_path, resumable=True)
            uploaded_file = service.files().create(body=file_metadata, media_body=media, fields="id").execute()
            print(f"Uploaded new file ID: {uploaded_file.get('id')}")
        return True
    except Exception as e:
        log_error(f"Failed to upload or update {file_name} to Google Drive: {e}")
        return False


# Fetch, decompress, and upload files
def fetch_and_upload_files():
    try:
        # Fetch the list of available files
        response = requests.get(PARLTRACK_DUMPS_URL)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract all .zst file URLs
        file_urls = [
            PARLTRACK_DUMPS_URL + link["href"]
            for link in soup.find_all("a", href=True)
            if link["href"].endswith(".zst")
        ]

        # Authenticate with Google Drive
        drive_service = authenticate_google_drive()

        # Process each file
        for file_url in file_urls:
            try:
                print(f"Processing file: {file_url}")

                # Fetch the compressed file
                response = requests.get(file_url, stream=True)
                response.raise_for_status()

                # Decompress Zstandard data
                dctx = zstd.ZstdDecompressor()
                decompressed = dctx.stream_reader(response.raw)

                # Parse JSON
                data = json.load(decompressed)

                # Save the file locally
                file_name = os.path.basename(file_url).replace(".json.zst", ".json")
                file_path = os.path.join("/tmp", file_name)
                with open(file_path, "w") as f:
                    json.dump(data, f)

                # Upload the file to Google Drive
                success = upload_to_drive(drive_service, file_path, file_name)

                # Clean up the local file
                os.remove(file_path)

                if success:
                    print(f"Finished processing: {file_name}")
                else:
                    log_error(f"Failed to process {file_name} from {file_url}")

            except Exception as e:
                log_error(f"Error processing file {file_url}: {e}")

    except Exception as e:
        log_error(f"Critical error fetching files: {e}")

# Log errors to a file
def log_error(message):
    with open(LOG_FILE_PATH, "a") as log_file:
        log_file.write(f"{datetime.now().isoformat()} - {message}\n")
    print(f"Logged error: {message}")


# Run the process
if __name__ == "__main__":
    try:
        # Read the secret from an environment variable (provided by GitHub Actions)
        secret = os.getenv("GOOGLE_CREDENTIALS")
        if not secret:
            raise ValueError("GOOGLE_CREDENTIALS secret is missing!")

       # Write the secret to a temporary file
        write_credentials(secret)

        # Fetch and upload files
        fetch_and_upload_files()

    finally:
        # Clean up credentials and print log file location
        if os.path.exists(CREDENTIALS_PATH):
            os.remove(CREDENTIALS_PATH)
        print(f"Log file saved at: {LOG_FILE_PATH}")

Using credentials from: /content/tmp/credentials.json
Processing file: https://parltrack.eu/dumps/ep_amendments.json.zst
